## Explicit data movement: `Channel` and `RemoteChannel`

Channels in Julia are constructs to explicitly exchange data between workers.

They implement `put!`, `take!`, `fetch`, `isready` and `wait` methods.

In [ ]:
# ?Channel

In [ ]:
ch = Channel{Int}(5) # a channel that can hold up to 5 integers

In [ ]:
isready(ch) # something in the channel?

In [ ]:
put!(ch, 3)

In [ ]:
isready(ch)

In [ ]:
take!(ch)

In [ ]:
isready(ch)

In [ ]:
put!(ch, 4)

In [ ]:
fetch(ch) # basically take without a bang

In [ ]:
take!(ch)

Be careful, `take!` and `put!` are blocking if the channel is empty or full!

In [ ]:
isready(ch)

In [ ]:
# take!(ch) if we execute this, while isready(ch) == false, the current Julia session will hang.

## Channels for inter-process data movement: `RemoteChannel`

* A `Channel` is local to a process. Worker 2 cannot directly refer to a `Channel` on worker 3 and vice-versa.


* A `RemoteChannel`, however, can put and take values across workers. A `RemoteChannel` can be thought of as a handle to a `Channel`.


* Any process with a reference to a `RemoteChannel` can put and take items from the channel. Data is automatically sent to (or retrieved from) the process a `RemoteChannel` is associated with.


* The process id, pid, associated with a `RemoteChannel` identifies the process where the backing store, i.e., the backing Channel exists.

In [ ]:
nworkers()

In [ ]:
addprocs(4)

In [ ]:
?RemoteChannel

In [ ]:
# creates a channel on the second worker process
# create a RemoteChannel handle to this channel on the master process
const mychannel = RemoteChannel(()->Channel{Int}(10), workers()[2])

In [ ]:
whohas("mychannel")

In [ ]:
# One could create a global constant mychannel everywhere
@everywhere const mychannel = $mychannel

In [ ]:
whohas("mychannel")

However, as we said many times before, one should generally try to avoid globals. The following is preferable.

In [ ]:
function do_something()
    rc = RemoteChannel(()->Channel{Int}(10)) # lives on the master
    @sync for p in workers()
        @spawnat p put!(rc, myid())
    end
    rc
end

r = do_something()

In [ ]:
isready(r)

In [ ]:
while isready(r)
    @show take!(r)
end

The ecosystem also contains a couple of tools, that make data transfer even simpler. See for example [ParallelDataTransfer.jl](https://github.com/ChrisRackauckas/ParallelDataTransfer.jl/).